# フロントエンド層

フロントエンド（Front-end）層は、DNNにおいて入力データ（波形やスペクトログラムなど）を処理する最初の層です。

## 1D畳み込み層

In [1]:
import torch
import torchaudio

wav, sr = torchaudio.load("assets/example1.ogg")
wav = wav.unsqueeze(0)
print('入力：', wav.shape)

conv_1d = torch.nn.Conv1d(
    in_channels=2,
    out_channels=32,
    kernel_size=5,
    stride=1,
    padding=2,  # stride=1, padding=(kernel_size - 1) // 2　にすれば、出力サイズが入力サイズと同じになる
    dilation=1,
    groups=1,
)
out = conv_1d(wav)
print('出力：', out.shape)



入力： torch.Size([1, 2, 3120768])
出力： torch.Size([1, 32, 3120768])


## Dilated-Conv-1D 

## SincNetのフロントエンド層